This notebook is useful only if the imdb extracted .tsv files are saved in db_files folder

**Loading data from original Dataset**

Here we take the original files extracted from [imdb api]("https://www.imdb.com/interfaces/") which are huge (35M entries just one of the six tables)

Entries in title_akas are indexed by *"titleId"*, this is the foreign key index for
all other tables about titles, which are indexed by *"tconst"*

we select entries where region == US
we select entries where rating is not null



In [2]:
import pandas as pd
# loading three tables
title_akas = pd.read_table("db_files/title_akas.tsv")
title_basic = pd.read_table("db_files/title_basic.tsv")
title_ratings = pd.read_table("db_files/title_ratings.tsv")

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_22424\122432313.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas = pd.read_table("db_files/title_akas.tsv")
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_22424\122432313.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basic = pd.read_table("db_files/title_basic.tsv")


In [3]:
# filter entries where region == US
def filter_us(table):
    return table[table.region == 'US']

title_akas = filter_us(title_akas)
title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1417215 entries, 5 to 35065524
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1417215 non-null  object
 1   ordering         1417215 non-null  int64 
 2   title            1417215 non-null  object
 3   region           1417215 non-null  object
 4   language         1417215 non-null  object
 5   types            1417215 non-null  object
 6   attributes       1417215 non-null  object
 7   isOriginalTitle  1417215 non-null  object
dtypes: int64(1), object(7)
memory usage: 97.3+ MB


In [4]:
# gather entries from title_akas indexed by titleId and select matching entries from title_basic table
title_basic = title_basic.loc[title_basic['tconst'].isin(title_akas.titleId)]
title_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1332407 entries, 0 to 9646975
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1332407 non-null  object
 1   titleType       1332407 non-null  object
 2   primaryTitle    1332407 non-null  object
 3   originalTitle   1332407 non-null  object
 4   isAdult         1332407 non-null  object
 5   startYear       1332407 non-null  object
 6   endYear         1332407 non-null  object
 7   runtimeMinutes  1332407 non-null  object
 8   genres          1332407 non-null  object
dtypes: object(9)
memory usage: 101.7+ MB


In [5]:
# gather entries from title_akas indexed by titleId and select matching entries from title_ratings table
title_ratings = title_ratings.loc[title_ratings['tconst'].isin(title_akas.titleId)]
title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490253 entries, 0 to 1283429
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         490253 non-null  object 
 1   averageRating  490253 non-null  float64
 2   numVotes       490253 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB


In [6]:
# join the three tables with index the new generated index (tt0000001 to tt9916764) result of match between titleId and tconst
df = title_akas.set_index("titleId").join(title_ratings.set_index("tconst")).join(title_basic.set_index("tconst"))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1417215 entries, tt0000001 to tt9916764
Data columns (total 17 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ordering         1417215 non-null  int64  
 1   title            1417215 non-null  object 
 2   region           1417215 non-null  object 
 3   language         1417215 non-null  object 
 4   types            1417215 non-null  object 
 5   attributes       1417215 non-null  object 
 6   isOriginalTitle  1417215 non-null  object 
 7   averageRating    545675 non-null   float64
 8   numVotes         545675 non-null   float64
 9   titleType        1413853 non-null  object 
 10  primaryTitle     1413853 non-null  object 
 11  originalTitle    1413853 non-null  object 
 12  isAdult          1413853 non-null  object 
 13  startYear        1413853 non-null  object 
 14  endYear          1413853 non-null  object 
 15  runtimeMinutes   1413853 non-null  object 
 16  genres       

In [7]:
df.head(50)

,ordering,title,region,language,types,attributes,isOriginalTitle,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0,5.7,1955.0,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0,5.8,263.0,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0,6.2,2591.0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0,6.2,2591.0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0,6.2,2591.0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,7,Blacksmithing,US,\N,\N,informal alternative title,0,6.2,2591.0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000006,3,Chinese Opium Den,US,\N,imdbDisplay,\N,0,5.1,177.0,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,1,Corbett and Courtney Before the Kinetograph,US,\N,imdbDisplay,\N,0,5.4,812.0,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000007,5,The Corbett-Courtney Fight,US,\N,alternative,\N,0,5.4,812.0,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000007,6,Jim Corbett vs. Peter Courtney,US,\N,alternative,\N,0,5.4,812.0,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


In [8]:
# filter entries where average rating != null
df = df.loc[df['averageRating'].notnull()]
df = df.loc[df['numVotes'].notnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 545675 entries, tt0000001 to tt9916428
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ordering         545675 non-null  int64  
 1   title            545675 non-null  object 
 2   region           545675 non-null  object 
 3   language         545675 non-null  object 
 4   types            545675 non-null  object 
 5   attributes       545675 non-null  object 
 6   isOriginalTitle  545675 non-null  object 
 7   averageRating    545675 non-null  float64
 8   numVotes         545675 non-null  float64
 9   titleType        545675 non-null  object 
 10  primaryTitle     545675 non-null  object 
 11  originalTitle    545675 non-null  object 
 12  isAdult          545675 non-null  object 
 13  startYear        545675 non-null  object 
 14  endYear          545675 non-null  object 
 15  runtimeMinutes   545675 non-null  object 
 16  genres           545675 non-null

In [10]:
import os
os.makedirs('db_files', exist_ok=True)
df.to_csv('db_files/db_cleaned.csv')